In [1]:
import pandas as pd
import itertools
import json
import sys
import numpy as np
import nltk

import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielbrai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielbrai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Aquisição e Transformação do Conjunto de Dados

A base de dados foi adquirida através do site [Scryfall](https://scryfall.com/docs/api/bulk-data), o qual disponibiliza um conjunto de arquivos contendo as informações de cada carta em formato json. Optou-se por trabalhar com o arquivo que agrega as informaćões das cartas apenas em inglês, dado que o idioma não afeta a análise a ser executada.

Uma vez que se tenha carregado o arquivo para um *dataframe* é importante olhar sua estrutura (isto é, conhecer suas colunas - ou *features*) para planejar como será executada sua manipulação.

In [2]:
data = pd.read_json('./default-cards-20220621090456.json')
data.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang',
       'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line',
       'oracle_text', 'power', 'toughness', 'colors', 'color_identity',
       'keywords', 'legalities', 'games', 'reserved', 'foil', 'nonfoil',
       'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id',
       'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri',
       'scryfall_set_uri', 'rulings_uri', 'prints_search_uri',
       'collector_number', 'digital', 'rarity', 'flavor_text', 'card_back_id',
       'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame',
       'full_art', 'textless', 'booster', 'story_spotlight', 'edhrec_rank',
       'penny_rank', 'prices', 'related_uris', 'all_parts', 'promo_types',
       'arena_id', 'preview', 'security_stamp', 'produced_mana', '

### Pré-processamento 0: Remoção de Colunas sem Valor e de Registros Duplicados ou Pouco Significativos
Conforme colocado na saída acima, verifica-se uma série de colunas na estrutura de dados na qual cada carta é armazenadas; a primeira estratégia adotada será eliminar aquelas que, numa primeira análise, não trazem informações significativas para se definir se uma carta pode ou não se tornar uma *staple* (uma carta muita requisitada pelos jogadores, a qual se torna uma peça fundamental dentro de um deck ou mesmo um formato de jogo). Neste primeiro momento, elencamos as seguintes colunas para serem removidas de nosso conjunto de dados:

In [3]:
columns_to_remove = ['mtgo_id', 'object', 'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 'released_at', 'uri', 
                     'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'foil', 'nonfoil',
                    'finishes', 'oversized', 'promo', 'variation', 'set_id', 'set_uri', 'set_search_uri',
                    'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'artist_ids',
                    'illustration_id', 'full_art', 'textless', 'booster', 'story_spotlight', 'penny_rank',
                    'related_uris', 'all_parts', 'promo_types', 'arena_id', 'preview', 'security_stamp', 'watermark',
                    'frame_effects', 'printed_name', 'card_faces', 'tcgplayer_etched_id', 'printed_type_line',
                    'printed_text', 'content_warning', 'variation_of', 'flavor_name', 'multiverse_ids', 'id',
                    'oracle_id', 'games', 'set', 'set_type', 'card_back_id', 'artist', 'border_color',
                    'frame', 'color_indicator', 'set_name', 'prices']

In [4]:
data_after_clean_columns = data.copy()
data_after_clean_columns.drop(columns=columns_to_remove, inplace=True)
data_after_clean_columns.columns

Index(['name', 'lang', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power',
       'toughness', 'colors', 'color_identity', 'keywords', 'legalities',
       'reserved', 'reprint', 'digital', 'rarity', 'flavor_text',
       'edhrec_rank', 'produced_mana', 'loyalty', 'life_modifier',
       'hand_modifier'],
      dtype='object')

Após a remoção das colunas tidas como pouco significantes, verifica-se uma estrutura mais enxuta, focada na características centrais da carta, conforme pode ser visto na saída anterior. Há três colunas que, embora não tragam informações consideradas relevantes, foram mantidas na estrutura para realizar uma filtragem das informações, a saber:
* **lang** - representa o idioma da carta; algumas cartas aparecem em mais de um idioma e, quando uma dessas versões é lançada exclusivamente em um mercado - como o japonês, como algumas cartas de *planeswalker* da coleção *War of the Spark*, ela aparece duplicada no dataset. Há casos em que uma coleção inteira é destinada a mercado consumidor, como a coleção *Portal Three Kingdoms*, o que faz com que tais cartas sejam pouco usadas dada a dificuldade em obtê-las, então também optou-se pela sua remoção. Recentemente, surgiram algumas varições de cartas, impresas numa versão no idioma **Phyrexiano**, uma língua fictícia, criada exclusivamente para o jogo; dada que se assemelha à primeira situação apresentada, estas cartas também foram removidas.
* **reprint** - essa informação vai apresentar se uma carta é ou não um _reprint_ - isto é, se ela foi reimpressa em alguma coleção após seu lançamento original; esta coluna seria representativa caso o foco do trabalho fosse predizer preços, mas dado que este não é o foco da proposta, sua remoção não será prejudicial.
* **digital** - deste de o lançamento da plataforma *Magic The Gathering Online* teve-se algumas coleções que foram lançadas exclusivamente para o ambiente digital. Isso tornou-se mais visível com o lançamento de um outro jogo virtual, o *Magic The Gathering Arena*, no qual surgiram cartas exclusivas para este meio. Dado que o foco do trabalho é o formato *Commander*, o qual é jogado exclusivamente no formato presencial, tais cartas exclusivas também serão removidas.

In [5]:
digital_content_index = data_after_clean_columns[data_after_clean_columns.digital == True].index
data_after_remove_digital_content = data_after_clean_columns.drop(digital_content_index)

In [6]:
reprints_index = data_after_remove_digital_content[data_after_remove_digital_content.reprint == True].index
data_after_remove_reprints = data_after_remove_digital_content.drop(reprints_index)

In [7]:
non_english_cards_index = data_after_remove_reprints[data_after_remove_reprints.lang != 'en'].index
data_after_remove_non_english_cards = data_after_remove_reprints.drop(non_english_cards_index)

In [8]:
auxiliar_columns = ['lang', 'reprint', 'digital']
data_pre_processed = data_after_remove_non_english_cards.drop(columns=auxiliar_columns)
data_pre_processed.columns

Index(['name', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power',
       'toughness', 'colors', 'color_identity', 'keywords', 'legalities',
       'reserved', 'rarity', 'flavor_text', 'edhrec_rank', 'produced_mana',
       'loyalty', 'life_modifier', 'hand_modifier'],
      dtype='object')

Após esse primeiro pré-processamento, chegou-se a um conjunto de dados inicial contendo 20 colunas, conforme colocado acima; este conjunto de dados ainda sofrerá novos processamentos, até que encontre-se uma versão adequada à proposta oferecida.

### Pré-processamento 1: Transformção da Coluna de Raridade
Cada carta do jogo pode perternce a uma de 4 possíveis raridades - comum, incomum rara ou mítica. Para que seja possível processá-lo, adtou-se a mesma abordagem aplicada às palavras-chave.

In [9]:
pd.get_dummies(data_pre_processed.rarity.explode())

,common,mythic,rare,special,uncommon
0,0,0,0,0,1
1,1,0,0,0,0
3,1,0,0,0,0
5,0,0,0,0,1
7,1,0,0,0,0
...,...,...,...,...,...
70076,1,0,0,0,0
70079,0,0,0,0,1
70080,1,0,0,0,0
70081,0,0,0,0,1


In [10]:
rarity_dummies = pd.get_dummies(data_pre_processed.rarity.explode())
rarity_dummies = rarity_dummies.groupby(rarity_dummies.index).sum()
data_step_1 = pd.concat([data_pre_processed,rarity_dummies],axis=1,ignore_index=False)

In [11]:
data_step_1.drop(columns=['rarity'], inplace=True)

### Pré-processamento 2: Transformção da Coluna de Cores e Identidade de Cor
Cada carta associa-se a uma ou mais cores, dentre as cinco disponíveis no universo do jogo; tal associação pode ser encontrada através da coluna **colors**, na forma de um array contendo todas as cores identificadas no custo de mana da referida carta. A transformação consistiu em alterar tal informação, tornando-a em um conjunto de _dummy variables_

In [12]:
colors_dummies = pd.get_dummies(data_step_1.colors.explode())
colors_dummies = colors_dummies.groupby(colors_dummies.index).sum()
data_step_2 = pd.concat([data_step_1,colors_dummies],axis=1,ignore_index=False)

In [13]:
data_step_2['colors_in_identity'] = data_step_2.color_identity.str.len()
data_step_2.drop(columns=['color_identity','colors', 'mana_cost'], inplace=True)

### Pré-processamento 3: Definição de "Staples" do Formato Commander

Essa foi a definição das duas classes possíveis pelas cartas - _**staple**_, cartas que possuem grande impacto no formato, e _**not_staple**_, que são cartas que para o formato em questão possuem menor relevância; essa etapa foi executada de forma arbitrária, definindo-se como parâmetro a coluna **edhrec_rank**, que armazena um valor numérico que representa a popularidade da carta dentro do site (EDHREC)[edhrec.com], que é especializado no formato de jogo Commander (quanto mais baixo o valor, mais provável que a carta seja relevante para o formato. 

Para os valores ausentes, fez-se o preenchimento com maior valor inteiro possível; a partir desse ponto, dividiu-se as amostras em dois grupos: para a classe positiva, assumiu-se que quaiquer amostras cujo valor da coluna analisada fosse menor que 10.000 pertenceriam a ela; para a classe negativa, todo o restante foi alocado nela.

In [14]:
data_step_3 = data_step_2.copy()
data_step_3.loc[data_step_3.edhrec_rank.isnull(), 'edhrec_rank'] = 99999
data_step_3['is_staple'] = np.where(data_step_3.edhrec_rank <= 10000, 1, 0)
data_step_3.drop(columns=['edhrec_rank'], inplace=True)

### Pré-processamento 4: Remoção de cartas ilegais no formato commander

Pensando na saúdo do formato e no balanceamento do jogo, a cada ciclo de lançamento é realizada uma análise do jogo e como novas cartas interagem com cartas pré-existentes, e em casos onde interações muito fortes possam desestabilizar o formato as cartas envolvidadas podem ser banidas - isto é, podem ser proibidas de serem utilizadas. 
Logo, como a proposta é trazer um modelo que identifique possíveis cartas relevantes ao formato, não faz sentido manter cartas banidas do mesmo presentes no _dataset_, e por isso optou-se pela sua remoção.

In [15]:
def get_index_from_illegal_commander_cards(row, stored_index):
    if row.legalities['commander'] != 'legal':
        stored_index.append(row.name)

In [16]:
data_step_4 = data_step_3.copy()
stored_index=[]
data_step_4.apply(lambda x: get_index_from_illegal_commander_cards(x, stored_index), axis=1)
data_step_4.drop(stored_index, inplace=True)
data_step_4.drop(columns=['legalities'], inplace=True)

### Pré-processamento 5: Substituição dos nomes das cartas nos textos de regras

É normal que os textos de regras, que serão os alvos primários do processamento por NLP, façam referências ao nome das próprias cartas; dado que esse comportamento pode produzir algum ruído para os modelos treinados, adotou-se a estratégia de padronizá-los através de um *token* de marcação, definido como _**CARDNAME**_. Com isso, o nome da carta em si torna-se irrelevante para os modelos treinados, embora o termo _**CARDNAME**_, em contrapartida, possua grande impacto, dado que é o sujeito da sentença apresentada.

Uma vez que se tenha implementado essa alteração, removeu-se a coluna de nome também, dado que ela não traz nenhum recurso adicional ao modelo.

In [17]:
def check_tokek_cardname(row):
    if row['name'] in row['oracle_text']:
        return row['oracle_text'].replace(row['name'], 'CARDNAME')
    else:
        return row['oracle_text']

In [18]:
data_step_5 = data_step_4.copy()
data_step_5.oracle_text.fillna('', inplace=True)
data_step_5['oracle_text_cardname'] = data_step_5.apply(check_tokek_cardname, axis=1)
data_step_5.drop(columns=['oracle_text', 'name'], inplace=True)

In [19]:
data_step_5.oracle_text_cardname
data_step_5.loc[data_step_5.oracle_text_cardname.isnull()]['oracle_text_cardname']
data_step_5.shape

(25715, 24)

### Pré-processamento 6: Correção da Informação de Produção de Mana

No conjunto de dados analisado, essa informação estava presente na forma de uma lista, contendo todas as possíveis cores de mana que uma carta poderia gerar; para o modelo proposto, é mais interessante que essa informação seja de caráter numérico; assim a abordagem adotada foi definir como dado a quantidade de cores possíveis que uma carta é capaz de gerar, que é representada pelo tamanho da lista nessa coluna.

In [20]:
def calculate_mana_factor(row):
    type(row.produced_mana)
    return len(row.produced_mana) if row.produced_mana else 0

In [21]:
data_step_6 = data_step_5.copy()
data_step_6.produced_mana.fillna(0, inplace=True)
data_step_6.produced_mana = data_step_6.apply(calculate_mana_factor, axis=1)
data_step_6.shape

(25715, 24)

### Pré-processamento 7: Transformção da Coluna de Habilidades
Várias habilidades dentro do jogo tornaram-se um padrão mecânico do mesmo (as chamadas _**evergreen keyword**_, dispensando explicações detalhadas e podendo aparecer em qualquer carta, lançada em qualquer coleção. Essas habilidades são representadas por palavras-chaves, as quais se encontram na coluna *keywords* no *dataset* trabalhado, na forma de uma lista de *strings*, onde cada entrada desta lista representa uma palavra-chave. A proposta é transformar essa coluna única em diversas colunas, onde cada palavra-chave se torne uma coluna, e seu valor seja 1, quando o registro o apresentar, ou 0, caso contrário - ou seja, uma _dummy variable_.

In [22]:
keywords_dummies = pd.get_dummies(data_step_6['keywords'].explode())
keywords_dummies = keywords_dummies.groupby(keywords_dummies.index).sum()
keywords_dummies.shape

(25715, 241)

In [23]:
data_step_7 = pd.concat([data_step_6,keywords_dummies],axis=1,ignore_index=False)

In [24]:
data_step_7.drop(columns=['keywords'], inplace=True)
data_step_7.shape

(25715, 264)

Ao realizar essa operação, um total de 253 novas colunas serão adicionadas ao nosso dataset, conforme demonstrado a seguir; há de se lembrar que, uma vez realizada a transformaćão, é necessário remover a coluna original que continha as palavras-chave das habilidades.

In [25]:
len(keywords_dummies.columns)

241

### Pré-processamento 8: Correção do Tipo de Carta

No _dataset_ trabalhado, os tipos de cartas continham todas as possibilidades presentes no jogo, o que implica em um grande volume de categorias pouco significativas. Para evitar tal efeito, optou-se pelo agrupamento das cartas em categorias mais genéricas, de modo a ter um nível mais signifiativo de informação.

In [26]:
def correct_card_type(row):
    types = []
    if 'Creature' in row.type_line:
        types.append('Creature')
    if 'Instant' in row.type_line:
        types.append('Instant')
    if 'Sorcery' in row.type_line:
        types.append('Sorcery')
    if 'Enchantment' in row.type_line:
        types.append('Enchantment')
    if 'Artifact' in row.type_line:
        types.append('Artifact')
    if 'Land' in row.type_line:
        types.append('Land')
    if 'Planeswalker' in row.type_line:
        types.append('Planeswalker')
    return types

In [27]:
data_step_8 = data_step_7.copy()
data_step_8.type_line.fillna('', inplace=True)
data_step_8.type_line = data_step_8.apply(correct_card_type, axis=1)

In [28]:
types_dummies = pd.get_dummies(data_step_8['type_line'].explode())
types_dummies = types_dummies.groupby(types_dummies.index).sum()
data_step_8 = pd.concat([data_step_8,types_dummies],axis=1,ignore_index=False)
data_step_8.drop(columns=['type_line'], inplace=True)

### Pré-processamento 9: Contabilização do Flavor Text 

O _flavor text_ não possui peso algum em relação às regras, e poderia ser eliminado sem quaisquer efeitos colaterais; todavia, ele traz uma informação implícita consigo: quanto maior o _flavor text_, menos espaço na caixa de texto para a adição de habilidades ou outras informações que afetem o funcionamento do jogo e possam permitir que a carta se torne relevante ao formato analisado.

In [29]:
def count_flavor_text(row):
    return len(row.flavor_text)

In [30]:
data_step_9 = data_step_8.copy()
data_step_9.flavor_text.fillna('', inplace=True)
data_step_9['flavor_text_count'] = data_step_9.apply(count_flavor_text, axis=1)
data_step_9.drop(columns=['flavor_text'], inplace=True)

### Pré-processamento 10: Correção de valores numéricos

Nas principais colunas que comportam valores numéricos - **cmc**, **power**, **toughness**, **loyalty**, **life_modifier** e **hand_modifier** - foi necessário assegurar que não haveria a ocorrência valores ausentes. 

Dentro da mecânica do jogo, algumas cartas podem ter seus atributos alterados de acordo com situações específicas - como a quantidade de cartas disponíveis no baralho do jogador ou o número de cartas de um determinado tipo em sua zona de descarte. Da mesma forma, alguns atributos de jogo, como a quantidade de cartas permitidas nas mãos ou o total de pontos de vida dos jogadores, também são afetados pelos efeitos de cartas diversas. Todos esses valores são representados por signos gráficos - por exemplo, uma carta grafada com o valor '\*+1' em seu campo de resistência (toughness) indica que aquela carta em questão possuí resistência igual a 1 adicionando-se uma quantidade qualquer que depende de um efeito dentro do jogo.

Por fim, algumas cartas possuem um custo de execução que também é dependente de condições específicas ou da vontade de seu jogador, sendo representadas pelo signo gráfico 'X'.

Para todas essas situações, fez-se o preenchimento dos valores ausentes pelo valor 0.


In [31]:
data_step_10 = data_step_9.copy()
data_step_10.cmc.fillna(0, inplace=True)
data_step_10.power.fillna(0, inplace=True)
data_step_10.toughness.fillna(0, inplace=True)
data_step_10.loyalty.fillna(0, inplace=True)
data_step_10.life_modifier.fillna(0, inplace=True)
data_step_10.hand_modifier.fillna(0, inplace=True)
data_step_10.dropna()

data_step_10.rename(columns = {'power':'power_value', 'toughness':'toughness_value'}, inplace = True)

In [32]:
def replace_asterisk(row, column):
    regexp_asterisk = re.compile(r'\*')
    regexp_x = re.compile(r'\b[xX]{1}')

    if regexp_asterisk.search(str(row[column])) or regexp_x.search(str(row[column])):
        return 0
    else:
        return row[column]

In [33]:
data_step_10.power_value = data_step_10.apply(lambda x: replace_asterisk(x, 'power_value'), axis=1)
data_step_10.toughness_value = data_step_10.apply(lambda x: replace_asterisk(x, 'toughness_value'), axis=1)
data_step_10.cmc = data_step_10.apply(lambda x: replace_asterisk(x, 'cmc'), axis=1)
data_step_10.loyalty = data_step_10.apply(lambda x: replace_asterisk(x, 'loyalty'), axis=1)
data_step_10.life_modifier = data_step_10.apply(lambda x: replace_asterisk(x, 'life_modifier'), axis=1)
data_step_10.hand_modifier = data_step_10.apply(lambda x: replace_asterisk(x, 'hand_modifier'), axis=1)

In [34]:
data_step_10.oracle_text_cardname
data_step_10.shape

(25715, 270)

### Pré-processamento 11: Remoção de Stopwords e Tokenização do Texto de Regras

A partir da coluna de texto de regras, realizou-se o processamento para remoção de _stopwords_ e para a quebra do texto em _tokens_.

In [35]:
data_step_11 = data_step_10.copy()
cv = CountVectorizer(stop_words='english')
cv_matrix = cv.fit_transform(data_step_11['oracle_text_cardname']) 

In [36]:
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=data_step_11.index, columns=cv.get_feature_names_out())
df_dtm

data_step_11 = pd.concat([data_step_11,df_dtm],axis=1,ignore_index=False)
data_step_11.drop(columns=['oracle_text_cardname'], inplace=True)
data_step_11

,cmc,power_value,toughness_value,reserved,produced_mana,loyalty,life_modifier,hand_modifier,common,mythic,...,zacama,zareth,zero,zhang,zndrsplt,zol,zombie,zombies,zone,zubera
0,6.0,3,3,False,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,2,2,False,0,0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,1,2,False,0,0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,2,1,False,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3.0,3,1,False,0,0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70071,3.0,1,1,False,0,0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
70076,0.0,0,0,False,6,0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
70079,4.0,0,0,False,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
70081,5.0,4,4,False,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Treinamento dos Modelos

In [37]:
features = data_step_11.columns.to_list()
print(f'is_staple está na lista: {"is_staple" in features}')
features.remove('is_staple')
print(f'is_staple está na lista: {"is_staple" in features}')
X = data_step_11.drop('is_staple', axis=1)
y = data_step_11['is_staple']

is_staple está na lista: True
is_staple está na lista: False


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

In [39]:
y.shape

(25715,)

In [40]:
model_LinearSVC = LinearSVC(max_iter=5000)
model_LinearSVC.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/d3top-magic/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=5000)

In [41]:
y_pred = model_LinearSVC.predict(X_test)
print(f'Acurácia do modelo de regressão logística: {accuracy_score(y_true=y_test, y_pred=y_pred)}')

Acurácia do modelo de regressão logística: 0.7438170788614092


In [42]:
model_LogisticRegression = LogisticRegression(random_state=0, n_jobs=4)
model_LogisticRegression.fit(X_train, y_train)

LogisticRegression(n_jobs=4, random_state=0)

In [43]:
y_pred = model_LogisticRegression.predict(X_test)
print(f'Acurácia do modelo de regressão logística: {accuracy_score(y_true=y_test, y_pred=y_pred)}')

Acurácia do modelo de regressão logística: 0.7497277959247162
